# Warning
This tutorial is a work in progress, it does not cover all the functionalities of the preprocessing package yet.

# preprocessing tutorial
In this Notebook, we will demonstrate how to preprocess brain MR images with the [BrainLes preprocessing package](https://github.com/BrainLesion/preprocessing/tree/main/brainles_preprocessing).
<!-- TODO elaborate -->

### Requirements
This tutorial requires:

* Python 3.10+ environment
* Docker
* Linux (Windows and Mac OS will be supported in future versions)
<!-- TODO specify version -->

optional (but recommended):
* CUDA
<!-- TODO specify -->





### Setup Colab environment
If you installed the packages and requirements on your own machine, you can skip this section and start from the import section. Otherwise you can follow and execute the tutorial on your browser. In order to start working on the notebook, click on the following button, this will open this page in the Colab environment and you will be able to execute the code on your own.

<a target="_blank" href="https://colab.research.google.com/github/BrainLesion/tutorials/blob/main/AURORA/tutorial.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

Now that you are visualizing the notebook in Colab, run the next cell to install the packages we will use. There are few things you should follow in order to properly set the notebook up:
1. Warning: This notebook was not authored by Google. Click on 'Run anyway'.
1. When the installation commands are done, there might be "Restart runtime" button at the end of the output. Please, click it.


## Installation of dependencies

In [1]:
%pip install brainles_preprocessing auxiliary
%pip install matplotlib

%load_ext autoreload
%autoreload 2

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


By running the next cell you are going to create a folder in your Google Drive. All the files for this tutorial will be uploaded to this folder. After the first execution you might receive some warning and notification, please follow these instructions:
1. Permit this notebook to access your Google Drive files? Click on 'Yes', and select your account.
Google Drive for desktop wants to access your Google Account. Click on 'Allow'.
1. At this point, a folder has been created and you can navigate it through the lefthand panel in Colab, you might also have received an email that informs you about the access on your Google Drive

In [2]:
# Create a folder in your Google Drive
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# Don't run this cell if you already cloned the repo
# !git clone https://github.com/BrainLesion/tutorials.git

In [4]:
# make files from the repo available in colab
import sys

COLAB_BASE_PATH = "/content/tutorials/BraTS-Toolkit/"
sys.path.insert(0, COLAB_BASE_PATH)

### Imports

## Raw Input data
TODO describe raw input data. Two exams from TCIA/TCGA with T1, T1c, T2, T2-FLAIR with skull etc.
TODO visualize it

## Preprocessing
BraTS challenge algorithms expect co-registered, skullstripped files in SRI-24 space, to achieve this preprocessing is required.
Instead of using the vanilla preprocessing pipeline from BraTS Toolkit, we recommend using the new [BrainLes preprocessing package](https://github.com/BrainLesion/preprocessing/tree/main/brainles_preprocessing).

<!-- For more details about preprocessing see the dedicated tutorial at TODO -->

First, we define a function that procoesses an exam.

In [5]:
import datetime

from auxiliary.normalization.percentile_normalizer import PercentileNormalizer
from auxiliary.turbopath import turbopath
from tqdm import tqdm

from brainles_preprocessing.brain_extraction import HDBetExtractor
from brainles_preprocessing.modality import Modality
from brainles_preprocessing.preprocessor import Preprocessor
from brainles_preprocessing.registration import NiftyRegRegistrator


def preprocess_exam_in_brats_style(inputDir: str) -> None:
    """
    Perform BRATS (Brain Tumor Segmentation) style preprocessing on MRI exam data.

    Args:
        inputDir (str): Path to the directory containing raw MRI files for an exam.

    Raises:
        Exception: If any error occurs during the preprocessing.

    Example:
        brat_style_preprocess_exam("/path/to/exam_directory")

    This function preprocesses MRI exam data following the BRATS style, which includes the following steps:
    1. Normalization using a percentile normalizer.
    2. Registration and correction using NiftyReg.
    3. Brain extraction using HDBet.

    The processed data is saved in a structured directory within the input directory.

    Args:
        inputDir (str): Path to the directory containing raw MRI files for an exam.

    Returns:
        None
    """
    inputDir = turbopath(inputDir)
    try:
        print("*** start ***")

        # where are the raw mr files?
        btk_raw_dir = turbopath(inputDir)

        # is the exam already processed?
        brainles_dir = turbopath(inputDir) + "/" + inputDir.name + "_brainles"
        raw_bet_dir = brainles_dir / "raw_bet"
        norm_bet_dir = brainles_dir / "normalized_bet"
        raw_skull_dir = brainles_dir / "raw_skull"
        norm_skull_dir = brainles_dir / "normalized_skull"

        # if not os.path.exists(prep_dir):
        # if os.path.exists(prep_dir):
        t1_file = btk_raw_dir.files("*t1.nii.gz")
        t1c_file = btk_raw_dir.files("*t1c.nii.gz")
        t2_file = btk_raw_dir.files("*t2.nii.gz")
        flair_file = btk_raw_dir.files("*fla.nii.gz")

        if len(t1_file) == len(t1c_file) == len(t2_file) == len(flair_file) == 1:
            # print(t1_file)
            # print(t1c_file)
            # print(t2_file)
            # print(flair_file)

            t1File = t1_file[0]
            t1cFile = t1c_file[0]
            t2File = t2_file[0]
            flaFile = flair_file[0]

            # normalizer
            percentile_normalizer = PercentileNormalizer(
                lower_percentile=0.1,
                upper_percentile=99.9,
                lower_limit=0,
                upper_limit=1,
            )

            # define modalities
            center = Modality(
                modality_name="t1c",
                input_path=t1cFile,
                raw_bet_output_path=raw_bet_dir / inputDir.name + "_t1c_bet_raw.nii.gz",
                raw_skull_output_path=raw_skull_dir / inputDir.name
                + "_t1c_skull_raw.nii.gz",
                normalized_bet_output_path=norm_bet_dir / inputDir.name
                + "_t1c_bet_normalized.nii.gz",
                normalized_skull_output_path=norm_skull_dir / inputDir.name
                + "_t1c_skull_normalized.nii.gz",
                atlas_correction=True,
                normalizer=percentile_normalizer,
            )

            moving_modalities = [
                Modality(
                    modality_name="t1",
                    input_path=t1File,
                    raw_bet_output_path=raw_bet_dir / inputDir.name
                    + "_t1_bet_raw.nii.gz",
                    raw_skull_output_path=raw_skull_dir / inputDir.name
                    + "_t1_skull_raw.nii.gz",
                    normalized_bet_output_path=norm_bet_dir / inputDir.name
                    + "_t1_bet_normalized.nii.gz",
                    normalized_skull_output_path=norm_skull_dir / inputDir.name
                    + "_t1_skull_normalized.nii.gz",
                    atlas_correction=True,
                    normalizer=percentile_normalizer,
                ),
                Modality(
                    modality_name="t2",
                    input_path=t2File,
                    raw_bet_output_path=raw_bet_dir / inputDir.name
                    + "_t2_bet_raw.nii.gz",
                    raw_skull_output_path=raw_skull_dir / inputDir.name
                    + "_t2_skull_raw.nii.gz",
                    normalized_bet_output_path=norm_bet_dir / inputDir.name
                    + "_t2_bet_normalized.nii.gz",
                    normalized_skull_output_path=norm_skull_dir / inputDir.name
                    + "_t2_skull_normalized.nii.gz",
                    atlas_correction=True,
                    normalizer=percentile_normalizer,
                ),
                Modality(
                    modality_name="flair",
                    input_path=flaFile,
                    raw_bet_output_path=raw_bet_dir / inputDir.name
                    + "_fla_bet_raw.nii.gz",
                    raw_skull_output_path=raw_skull_dir / inputDir.name
                    + "_fla_skull_raw.nii.gz",
                    normalized_bet_output_path=norm_bet_dir / inputDir.name
                    + "_fla_bet_normalized.nii.gz",
                    normalized_skull_output_path=norm_skull_dir / inputDir.name
                    + "_fla_skull_normalized.nii.gz",
                    atlas_correction=True,
                    normalizer=percentile_normalizer,
                ),
            ]

            preprocessor = Preprocessor(
                center_modality=center,
                moving_modalities=moving_modalities,
                registrator=NiftyRegRegistrator(),
                brain_extractor=HDBetExtractor(),
                # optional: we provide a temporary directory as a sandbox for the preprocessin
                temp_folder="temporary_directory",
                limit_cuda_visible_devices="0",
            )

            preprocessor.run(
                save_dir_coregistration=brainles_dir + "/co-registration",
                save_dir_atlas_registration=brainles_dir + "/atlas-registration",
                save_dir_atlas_correction=brainles_dir + "/atlas-correction",
                save_dir_brain_extraction=brainles_dir + "/brain-extraction",
            )

    except Exception as e:
        print("error: " + str(e))
        print("conversion error for:", inputDir)

        time = str(datetime.datetime.now().time())

        print("** finished:", inputDir.name, "at:", time)

now we loop through the exams to preprocess the data:

In [6]:
EXAMPLE_DATA_DIR = turbopath("data")

exams = EXAMPLE_DATA_DIR.dirs()

for exam in tqdm(exams):
    print("processing:", exam)
    preprocess_exam_in_brats_style(exam)

  0%|          | 0/2 [00:00<?, ?it/s]

processing: /home/florian/flow/BrainLesion/tutorials/BraTS-Toolkit/data/OtherEXampleFromTCIA
*** start ***
File: /home/florian/flow/BrainLesion/tutorials/BraTS-Toolkit/tempo/atlas-space/atlas__t1c.nii.gz
preprocessing...
image shape after preprocessing:  (103, 160, 160)
prediction (CNN id)...
0
1
2
3
4
exporting segmentation...


 50%|█████     | 1/2 [03:57<03:57, 237.19s/it]

processing: /home/florian/flow/BrainLesion/tutorials/BraTS-Toolkit/data/TCGA-DU-7294
*** start ***
File: /home/florian/flow/BrainLesion/tutorials/BraTS-Toolkit/tempo/atlas-space/atlas__t1c.nii.gz
preprocessing...
image shape after preprocessing:  (103, 160, 160)
prediction (CNN id)...
0
1
2
3
4
exporting segmentation...


100%|██████████| 2/2 [12:10<00:00, 365.38s/it]
